In [1]:
# Clone the repository
!git clone https://github.com/fridamarieLJ/Materials-Design-with-Machine-Learning.git

# Navigate to the repository
%cd 'Materials-Design-with-Machine-Learning'

# List files in the repository
!ls

fatal: destination path 'Materials-Design-with-Machine-Learning' already exists and is not an empty directory.
/content/Materials-Design-with-Machine-Learning
Albert		 neural_network.ipynb  project_dataload.py	       __pycache__
Fingerprints.py  new_notebook.ipynb    project_gaussian.py	       Sinematrix.py
GP.py		 PCA_decomposition.py  project_main.ipynb	       SOAP.py
LoadData.py	 project_data	       project_main_test_albert.ipynb


In [8]:
# Import dependencies
!pip install ase
!pip install dscribe

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 84.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.4/117.4 kB 11.0 MB/s eta 0:00:00


In [9]:
# import self made functions
from project_dataload import data_load, check_data, summarize_1
from project_gaussian import *
from Fingerprints import Sinemats, Coulombmatrices, Ewaldsummatrices
from PCA_decomposition import *
from GP import GP
